In [2]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
from tensorflow.python.ops import control_flow_ops

In [3]:
BATCH_SIZE = 100
BATCH_NUM = 100
FEATURE_SIZE = 100
EMBEDDED_SIZE = 3
NUM_CHANNELS = 1
CLASS_SIZE = 100
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.99
TRAIN_SIZE = 100

In [4]:
def random_shuffle(data, label):
    idx = np.random.permutation(len(data))
    random_data, random_label = data[idx], label[idx]
    return random_data, random_label

def preprocess_label(label, size):
    label_one_hot = np.zeros(size)
    label_one_hot[label] = 1
    return label_one_hot
    
def preprocess_labels(labels, size):
    labels_one_hot = []
    for label in labels:
        label_one_hot = np.zeros(size)
        label_one_hot[label] = 1
        labels_one_hot.append(label_one_hot)
    return labels_one_hot
    
def generate_input_list(ent_embedding, rel_embedding, data_id):
    hrs = []
    rts = []
    label_hrs = []
    label_rts = []
    zero_array = np.zeros(100)
    for triplet_id in data_id:
        h = ent_embedding[triplet_id[0]]
        r = rel_embedding[triplet_id[2]]
        t = ent_embedding[triplet_id[1]]
        hr = np.transpose([[h, r, zero_array]])
        rt = np.transpose([[zero_array, r, t]])
        hrs.append(hr)
        rts.append(rt)
        label_hrs.append(triplet_id[1])
        label_rts.append(triplet_id[1])
    input_list = np.concatenate((hrs, rts), axis=0)
    label_list = np.concatenate((label_hrs, label_rts), axis=0)
    return input_list, label_list

In [4]:
def model():
    '''Create the Model
    '''
    
    #input
    # []: dimension
    
    x = tf.placeholder(tf.float32, [BATCH_SIZE,
                                   FEATURE_SIZE,
                                   EMBEDDED_SIZE,
                                   NUM_CHANNELS],
                          name = 'x-input')
    # x_hat = tf.placeholder(tf.float32, [BATCH_SIZE,
    #                                FEATURE_SIZE,
    #                                EMBEDDED_SIZE,
    #                                NUM_CHANNELS],
    #                       name = 'x_hat-input')

    y_gt = tf.placeholder(tf.float32, [None, CLASS_SIZE], name = 'y-input')
    is_training = tf.placeholder(tf.bool, [])

    
    #model
    filter_sizes = [2, 3, 4]
    with tf.variable_scope('branch-x'):
        with slim.arg_scope([slim.conv2d],
                           activation_fn=tf.nn.elu,
                           normalizer_fn=slim.batch_norm,
                           normalizer_params={'is_training': is_training, 'decay':0.95}):
            conv1 = slim.conv2d(x, 5, [1, 1], scope='conv1', padding='SAME')
            print(conv1)
            conv2_outputs = []
            for i, filter_size in enumerate(filter_sizes):
                # Convolution Layer
                name_scope = "conv2-%s" % filter_size
                kernel_size = [filter_size, 2]
                conv2 = slim.conv2d(conv1, 128, kernel_size, scope = name_scope, padding='SAME')
                print(conv2)
                conv2_outputs.append(conv2)
            conv2 = slim.conv2d(conv1, 128, [1, 1], scope = "conv2-1", padding='SAME')
            print(conv2)
            conv2_outputs.append(conv2)
            conv2_concat = tf.concat(conv2_outputs, 3)
            print(conv2_concat)

            conv3_outputs = []
            for i, filter_size in enumerate(filter_sizes):
                # Convolution Layer
                name_scope = "conv3-%s" % filter_size
                kernel_size = [filter_size, 2]
                conv3 = slim.conv2d(conv2_concat, 256, kernel_size, scope = name_scope, padding='VALID')
                pool = slim.max_pool2d(conv3, [FEATURE_SIZE - filter_size - 1, 0], scope = name_scope, padding='VALID')
                print(pool)
                conv3_outputs.append(pool)
            conv3 = slim.conv2d(conv2_concat, 256, [1, 1], scope = "conv3-1", padding='SAME')
            pool = slim.max_pool2d(conv3, [FEATURE_SIZE - 2, 0], scope = name_scope, padding='VALID')

            print(pool)
            conv3_outputs.append(pool)
            conv3_concat = tf.concat(conv3_outputs, 3)
            print(conv3_concat)

    with slim.arg_scope([slim.conv2d],
                           activation_fn=tf.nn.elu,
                           normalizer_fn=slim.batch_norm,
                           normalizer_params={'is_training': is_training, 'decay':0.95}):
        conv4 = slim.conv2d(conv3_concat, CLASS_SIZE, [1, 1], scope = "conv4", padding = 'SAME')
        pool = slim.max_pool2d(conv4, [2, 2], scope = "conv4", padding = 'VALID')
        logits = tf.reduce_mean(pool, axis = [1, 2])
        print(pool)
        print(logits)
        
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y_gt, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_gt, logits=logits))
    loss = cross_entropy
#     , dtype=np.int32
    step = tf.get_variable("step", [], initializer=tf.constant_initializer(0.0), trainable=False)
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, step, BATCH_NUM, 
                                               LEARNING_RATE_DECAY, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
    train_op = slim.learning.create_train_op(loss, optimizer, global_step=step)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    if update_ops:
        updates = tf.group(*update_ops)
        loss = control_flow_ops.with_dependencies([updates], loss)

    #Summary
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)
    merged_summary_op = tf.summary.merge_all()
    print(merged_summary_op)

    
    return{'x': x,
       'y_gt': y_gt,
       'is_training': is_training,
       'train_op': train_op,
       'global_step': step,
       'accuracy': accuracy,
       'loss': loss,
       'summary': merged_summary_op}

In [ ]:
def train():
    #loading data
    ent_embedding = np.loadtxt('./data/ent_embeddings_transE.txt', delimiter=',')
    rel_embedding = np.loadtxt('./data/rel_embeddings_transE.txt', delimiter=',')
    train_id = np.loadtxt('./data/triple2id.txt', dtype = np.int32)
    valid_id = np.loadtxt('./data/valid2id.txt', dtype = np.int32)

    train_data, train_label = generate_input_list(ent_embedding, rel_embedding, train_id)
    valid_data, valid_label = generate_input_list(ent_embedding, rel_embedding, valid_id)
    
    train_data, train_label = random_shuffle(train_data, train_label)
    valid_data, valid_label = random_shuffle(valid_data, valid_label)
    
    global TRAIN_SIZE, FEATURE_SIZE, CLASS_SIZE, BATCH_SIZE, BATCH_NUM, EPOCH_SIZE
    TRAIN_SIZE = train_data.shape[0]
    FEATURE_SIZE = train_data.shape[1]
    CLASS_SIZE = ent_embedding.shape[0]
    BATCH_SIZE = 1
    BATCH_NUM = TRAIN_SIZE/BATCH_SIZE + 1
    EPOCH_SIZE = 50
    
    #load model
    net = model()
       
    #start training
    with tf.Session() as sess:
        #write log
        summary_writer = tf.summary.FileWriter('./log', sess.graph)
        
        #initial the variables
        tf.global_variables_initializer().run()
        
        #Train cycle
        for epoch in range(EPOCH_SIZE):
            offset = 0
            for i in range(BATCH_NUM):
                offset = i * BATCH_SIZE
                data_batch = train_data[offset:min((offset + BATCH_SIZE), TRAIN_SIZE)]
                label_batch = preprocess_labels(train_label[offset:min((offset + BATCH_SIZE), TRAIN_SIZE)], ent_embedding.shape[0])
                train_dict = {net['x']: data_batch,
                             net['y_gt']: label_batch,
                             net['is_training']: True}
                step, _ = sess.run([net['global_step'], net['train_op']], feed_dict = train_dict)
                print('sucess!')
                break
                if step % 50 == 0:
                    loss, acc, summary = sess.run([net['loss'], net['accuracy'],net['summary']], feed_dict = train_dict)
                    print('Train step{}; loss{}; accuracy{}'.format(step, loss, acc))
                    summary_writer.add_summary(summary, step)
                
    

In [ ]:
train()

Tensor("branch-x/conv1/Elu:0", shape=(256, 100, 3, 5), dtype=float32)
Tensor("branch-x/conv2-2/Elu:0", shape=(256, 100, 3, 128), dtype=float32)
Tensor("branch-x/conv2-3/Elu:0", shape=(256, 100, 3, 128), dtype=float32)
Tensor("branch-x/conv2-4/Elu:0", shape=(256, 100, 3, 128), dtype=float32)
Tensor("branch-x/conv2-1/Elu:0", shape=(256, 100, 3, 128), dtype=float32)
Tensor("branch-x/concat:0", shape=(256, 100, 3, 512), dtype=float32)
Tensor("branch-x/conv3-2_1/MaxPool:0", shape=(256, 2, 2, 256), dtype=float32)
Tensor("branch-x/conv3-3_1/MaxPool:0", shape=(256, 2, 2, 256), dtype=float32)
Tensor("branch-x/conv3-4_1/MaxPool:0", shape=(256, 2, 2, 256), dtype=float32)
Tensor("branch-x/conv3-4_2/MaxPool:0", shape=(256, 2, 2, 256), dtype=float32)
Tensor("branch-x/concat_1:0", shape=(256, 2, 2, 1024), dtype=float32)
Tensor("conv4_1/MaxPool:0", shape=(256, 1, 1, 14951), dtype=float32)
Tensor("Mean:0", shape=(256, 14951), dtype=float32)
Tensor("Merge/MergeSummary:0", shape=(), dtype=string)


In [7]:
ent_embedding = np.loadtxt('./data/ent_embeddings_transE.txt', delimiter=',')
rel_embedding = np.loadtxt('./data/rel_embeddings_transE.txt', delimiter=',')
train_id = np.loadtxt('./data/triple2id.txt', dtype = np.int32)
train_data, train_label = generate_input_list(ent_embedding, rel_embedding, train_id)
data_batch = train_data[:256]


NameError: name 'data' is not defined

In [8]:
print(data_batch.shape)

(256, 100, 3, 1)
